## Importing The Required Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import gensim
import string
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l1
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score 
import numpy as np
from time import time
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Loading The Dataset.

In [ ]:
data= pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv', encoding='latin-1')
data

We are only going to use the first two columns also we are going to replace spam with 1 and ham with 0 in column 1.

In [ ]:
data=data.iloc[:,:2]
data['message_len'] = data.v2.apply(len)
data['v1']=data['v1'].replace({'ham':0,'spam':1})
data.head()

# Lets Do Some EDA.

Lets checkout the message length whether the sms is a spam or not.

In [ ]:
plt.figure(figsize=(12, 8))

data[data.v1==0].message_len.plot(bins=35, kind='hist', color='blue', 
                                       label='Ham messages', alpha=0.6)
data[data.v1==1].message_len.plot(kind='hist', color='red', 
                                       label='Spam messages', alpha=0.6)
plt.legend()
plt.xlabel("Message Length")

# Text Preprocessing.

In [ ]:
df=data.iloc[:,:2]
df.head()

In [ ]:
def clean_text(text):
 
    text = re.sub('[^a-zA-Z]', ' ', text)  

    text = text.lower()  

    text = text.split(' ')  
    
    text = [w for w in text if not w in set(stopwords.words('english'))] 

    text = ' '.join(text)    
            
    return text



df['v2'] = df['v2'].apply(lambda x : clean_text(x))
df.head()

## GloVe for Vectorization

Here we will use GloVe pretrained corpus model to represent our words.It is available in 3 varieties :50D ,100D and 200 Dimentional.We will try 50 D here because the Dataset is small.If you have a larger dataset,you can use 100D or 200D.

In [ ]:
def create_corpus(df):
    corpus=[]
    for tweet in tqdm(df['v2']):
        words=[word.lower() for word in word_tokenize(tweet) if((word.isalpha()==1) & (word not in stop))]
        corpus.append(words)
    return corpus

corpus=create_corpus(df)

In [ ]:
embedding_dict={}
with open('/kaggle/input/glove-global-vectors-for-word-representation/glove.6B.50d.txt','r') as f:
    for line in f:
        values=line.split()
        word=values[0]
        vectors=np.asarray(values[1:],'float32')
        embedding_dict[word]=vectors
f.close()

In [ ]:
MAX_LEN=10
tokenizer_obj=Tokenizer()
tokenizer_obj.fit_on_texts(corpus)
sequences=tokenizer_obj.texts_to_sequences(corpus)

tweet_pad=pad_sequences(sequences,maxlen=MAX_LEN,truncating='post',padding='post')

In [ ]:
word_index=tokenizer_obj.word_index
print('Number of unique words:',len(word_index))

In [ ]:
num_words=len(word_index)+1
embedding_matrix=np.zeros((num_words,50))

for word,i in tqdm(word_index.items()):
    if i > num_words:
        continue
    
    emb_vec=embedding_dict.get(word)
    if emb_vec is not None:
        embedding_matrix[i]=emb_vec

# Splitting The Data into train and validation set.

In [ ]:
X_train,X_val, y_train, y_val = train_test_split(tweet_pad,df.v1, test_size=.2, random_state=2)

In [ ]:
print('Shape of train sequences: ',X_train.shape)
print('Shape of train labels: ',y_train.shape)
print("Shape of Validation sequences: ",X_val.shape)
print("Shape of Validation  labels: ",y_val.shape)

# Creating Our Model.

In [ ]:
model=Sequential()

embedding_layer=Embedding(num_words,50,embeddings_initializer=Constant(embedding_matrix),
                   input_length=MAX_LEN,trainable=False)

model.add(embedding_layer)
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2,return_sequences=True))
model.add(tf.keras.layers.LSTM(32,return_sequences=True))
model.add(tf.keras.layers.LSTM(16))
model.add(tf.keras.layers.Dense(16, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

In [ ]:
optimzer=Adam(learning_rate=1e-4)
model.compile(loss='binary_crossentropy',optimizer=optimzer,metrics=['acc'])
model.summary()

## Fitting The Model.

In [ ]:
history=model.fit(X_train,y_train,batch_size=32,epochs=10,validation_data=(X_val,y_val),verbose=1)

## Creating a Dataframe of Loss And Accuracy.

In [ ]:
model_loss = pd.DataFrame(model.history.history)
model_loss.head()

## Lets visualize the Loss over the Epochs.

In [ ]:
model_loss[['loss','val_loss']].plot(ylim=[0,1])
plt.show()

## Lets visualize the Accuracy over the Epochs.

In [ ]:
model_loss[['acc','val_acc']].plot(ylim=[0,1])
plt.show()

# Thankyou For Reading!!!